# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

In [1]:
!pip install colabcode
from colabcode import ColabCode

ModuleNotFoundError: ignored

In [2]:
# Imports. Feel free to add or remove as necessary 

!pip install -Uq tensorflow-datasets==4.5.2
!pip install tf-nightly -q
import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

     |████████████████████████████████| 4.2 MB 8.8 MB/s 
     |████████████████████████████████| 583.3 MB 23 kB/s 
     |████████████████████████████████| 439 kB 62.0 MB/s 
     |████████████████████████████████| 5.9 MB 40.2 MB/s 
     |████████████████████████████████| 1.7 MB 46.4 MB/s 


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [4]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'
!cd {dir_drive_code}
os.chdir(dir_drive_code)

Mounted at /content/drive


In [27]:
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [29]:
data_processor.create_labels_df(dir_labels_csv)


Broken            species     age   test    val
sample_name    object_num                                                   
MV1012-BC-12   1.0         unbroken          NOT FORAM  1994.0  False  False
               2.0         unbroken   suggrunda eckisi  1994.0  False  False
               3.0           broken    bulimina exilis  1994.0  False  False
               4.0         unbroken  nonionella stella  1994.0  False  False
               5.0         unbroken          NOT FORAM  1994.0  False   True
...                             ...                ...     ...    ...    ...
MV1012-BC-9-10 165.0       unbroken  nonionella stella  1998.4  False  False
               166.0       unbroken  nonionella stella  1998.4  False  False
               167.0         broken          NOT FORAM  1998.4  False  False
               168.0       unbroken          NOT FORAM  1998.4  False  False
               169.0       unbroken          NOT FORAM  1998.4  False   True

[10999 rows x 5 columns]

In [30]:
# Collect label/metadata information from master CSV

labels_csv_path = os.path.join(dir_drive_data, 'Final_Diversity_CSVs', 'ML_master2.csv')
print(labels_csv_path)
assert os.path.isfile(labels_csv_path), 'Unable to find master CSV document!'
labels_df = pd.read_csv(labels_csv_path)
labels_df = labels_df.set_index(['sample_name', 'object_num'])
labels_df.sort_index()
# ACCESS via: labels_df.loc[('MV1012-BC-2', 1)]
labels_df = labels_df.dropna(how='any')

# Randomly decide if it will be in the train, val or test section.
# 0.8 training, 0.2 test
# 0.8 * 0.8 = 0.64 train, 0.16 val
sample_ind = np.random.random_sample(labels_df.shape[0])
labels_df['test'] = sample_ind > 0.8
labels_df['val'] = (sample_ind <= 0.8) & (sample_ind > 0.64)
labels_df.head()

NameError: ignored

In [ ]:
#Run data processing

assert os.path.isdir(out_dir), 'unable to find the output for processed training data'

imgs_dir = os.path.join(drive_dir_path, 'Box_Core_images')


for sample_dir in os.listdir(imgs_dir):
    sample_dir_path = os.path.join(imgs_dir, sample_dir)
    sample_name = re.match(r'(.+)_identify', sample_dir).groups()[0]
    # process_sample_dir(sample_dir_path, sample_name, labels_df, data_dir=dev_data_dir, override=False)
    process_sample_dir(sample_dir_path, sample_name, labels_df, data_dir=out_dir, override=False)


    
 

## Create tensorflow dataset and push to google cloud

In [ ]:
#@title constants
image_size = (416, 416)

input_shape = (*image_size, 3)
drive_prefix = '/content/drive/MyDrive/MV1012_SBB_images/ML_projects/'
 

logdir_base = f'{drive_prefix}/Training_logs/'
trained_model_dir_base = f'{drive_prefix}/Trained_models/'

batch_size = 128 * 2
google_cloud_storage = 'gs://paleo-ml'

In [ ]:

# TODO create the respective sets by IMPORTING the image dataset!
os.chdir(tfds_dir)
# print(os.getcwd())
import forams
training_set = tfds.load('forams', split='train')
validation_set = tfds.load('forams', split='val')
testing_set = tfds.load('forams', split='test')

In [ ]:
# tf.data.experimental.save(training_set, f'{google_cloud_storage}/training{batch_size}')
# tf.data.experimental.save(validation_set, f'{google_cloud_storage}/validation{batch_size}')
# tf.data.experimental.save(testing_set, f'{google_cloud_storage}/testing{batch_size}')
tf.data.Dataset.save(training_set, f'{google_cloud_storage}/training{batch_size}')
tf.data.Dataset.save(validation_set, f'{google_cloud_storage}/validation{batch_size}')
tf.data.Dataset.save(testing_set, f'{google_cloud_storage}/testing{batch_size}')